In [14]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.impute import KNNImputer
df=pd.read_csv("churn.csv")
df=df.drop("customerID",axis=1)
print(df.shape)
df=df.drop(df[df['TotalCharges'] == ' '].index)
df["SeniorCitizen"].replace(0, 'No', inplace=True)
df["SeniorCitizen"].replace(1, 'Yes', inplace=True)
df["TotalCharges"]=df["TotalCharges"].astype("float64")
print(df.dtypes)
#np.where(df.applymap(lambda x: x == ' '))

(7043, 20)
gender               object
SeniorCitizen        object
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object


In [26]:
for  variable in df.columns:
  print(variable,len(df[variable].unique()))
Dataset=df.replace('No phone service', np.nan)
Dataset=df.replace('No internet service', np.nan)
number_columns=["tenure","MonthlyCharges","TotalCharges"]
columns=[string for string in Dataset.columns if string not in number_columns]
print(Dataset.head())

gender 2
SeniorCitizen 2
Partner 2
Dependents 2
tenure 72
PhoneService 2
MultipleLines 3
InternetService 3
OnlineSecurity 3
OnlineBackup 3
DeviceProtection 3
TechSupport 3
StreamingTV 3
StreamingMovies 3
Contract 3
PaperlessBilling 2
PaymentMethod 4
MonthlyCharges 1584
TotalCharges 6530
Churn 2
   gender SeniorCitizen Partner Dependents  tenure PhoneService  \
0  Female            No     Yes         No       1           No   
1    Male            No      No         No      34          Yes   
2    Male            No      No         No       2          Yes   
3    Male            No      No         No      45           No   
4  Female            No      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity OnlineBackup  \
0  No phone service             DSL             No          Yes   
1                No             DSL            Yes           No   
2                No             DSL            Yes          Yes   
3  No phone service             D

In [25]:
import pickle
encoder=LabelEncoder()
def encode(data):
    '''function to encode non-null data and replace it in the original data'''
    #retains only non-null values
    nonulls = np.array(data.dropna())
    #reshapes the data for encoding
    impute_reshape = nonulls.reshape(-1,1)
    #encode date
    impute_ordinal = encoder.fit_transform(impute_reshape)
    #Assign back encoded values to non-null values
    data.loc[data.notnull()] = np.squeeze(impute_ordinal)
    return data
for column in columns:
    encode(Dataset[column])
pickle.dump(encoder,open("encoder.pkl","wb"))
print(Dataset)

     gender SeniorCitizen Partner Dependents  tenure PhoneService  \
0         0             0       1          0       1            0   
1         1             0       0          0      34            1   
2         1             0       0          0       2            1   
3         1             0       0          0      45            0   
4         0             0       0          0       2            1   
...     ...           ...     ...        ...     ...          ...   
7038      1             0       1          1      24            1   
7039      0             0       1          1      72            1   
7040      0             0       1          1      11            0   
7041      1             1       1          0       4            1   
7042      1             0       0          0      66            1   

     MultipleLines InternetService OnlineSecurity OnlineBackup  \
0                1               0              0            1   
1                0               0     

c:\Users\SABA\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\SABA\AppData\Local\Temp\ipykernel_21940\27966067.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[data.notnull()] = np.squeeze(impute_ordinal)
c:\Users\SABA\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\SABA\AppData\Local\Temp\ipykernel_21940\279

In [21]:
imputer=KNNImputer()
encode_data = pd.DataFrame(np.round(imputer.fit_transform(Dataset)),columns = Dataset.columns)

from sklearn.model_selection import train_test_split
X=encode_data.drop("Churn",axis=1)
y=encode_data["Churn"]
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.15,random_state=42,shuffle=True)

In [22]:
##Scaling data
scaler_x=StandardScaler()
Xtrain=scaler_x.fit_transform(X_train)
Xtest=scaler_x.transform(X_test)

In [23]:
##Descision Tree
import pickle
from sklearn import tree
from sklearn import metrics
from sklearn.model_selection import GridSearchCV,cross_val_score
from sklearn.metrics import accuracy_score
params = {'max_depth': list(range(1, 4)), 'max_leaf_nodes': [2,3,4],'criterion':['gini','entropy','log_loss']}
decisionTree = GridSearchCV(tree.DecisionTreeClassifier(random_state=42), params, verbose=0, cv=3)
decisionTree.fit(Xtrain ,y_train)
pickle.dump(decisionTree,open("DecisionTree.pkl","wb"))
treepred_train=decisionTree.predict(Xtrain)
treepred_test=decisionTree.predict(Xtest)
scores = cross_val_score(decisionTree, Xtrain, y_train, cv =3,scoring="accuracy")
##calculate cross validation score to evaluate the model
print("Cross Validation Scores: ", scores.mean())

Cross Validation Scores:  0.7845074057729496


In [24]:
###ANN
from sklearn.svm import SVC

svm_poly_reg = SVC(kernel="rbf", degree=3, C=1,  gamma="scale")

params = {'degree': list(range(4, 10)), 'C': [0.01,1,10,20]}
SVC = GridSearchCV(svm_poly_reg, params, cv=5,scoring='accuracy')
SVC.fit(Xtrain,y_train.ravel())
reg_pred_train=SVC.predict(Xtrain)
reg_pred_test=SVC.predict(Xtest)

In [26]:
scores = cross_val_score(SVC, Xtrain, y_train, cv =3,scoring="accuracy")
##calculate cross validation score to evaluate the model
print("Cross Validation Scores: ", scores.mean())

Cross Validation Scores:  0.798727860093997


In [8]:
Dataset['PaymentMethod'][4]



'Electronic check'